In [1]:
import pickle as p
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import os
import numpy as np
import prody as pd

In [3]:
alascan = p.load( open( "save.p", "rb" ) )


AESOP: Analysis of Electrostatic Structure of Proteins

Reed E. S. Harrison, Rohith R. Mohan, Dimitrios Morikis
University of California, Riverside; Department of Bioengineering

Correspondence should be directed to Prof. Dimitrios Morikis at
dmorikis@ucr.edu

Copyright (C) 2016  Reed E. S. Harrison, Rohith R. Mohan, Ronald D.
Gorham Jr., Chris A. Kieslich, Dimitrios Morikis

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
ERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.



In [4]:
scan = alascan
AA_dict = {
    'CYS': 'C',
    'ASP': 'D',
    'SER': 'S',
    'GLN': 'Q',
    'LYS': 'K',
    'ILE': 'I',
    'PRO': 'P',
    'THR': 'T',
    'PHE': 'F',
    'ASN': 'N',
    'GLY': 'G',
    'HIS': 'H',
    'LEU': 'L',
    'ARG': 'R',
    'TRP': 'W',
    'ALA': 'A',
    'VAL': 'V',
    'GLU': 'E',
    'TYR': 'Y',
    'MET': 'M'
}

In [5]:
pdbfile = os.path.join(scan.jobdir,
                           scan.pqr_complex_dir,
                           'wt.pqr')

In [6]:
selstr = scan.selstr
jobdir = scan.jobdir
title=''
dpi=300
cutoff=5.
E=2.5
node_size=1500
font_size=12
alpha=0.8
edge_color='g'
edge_width=3.

In [7]:
if len(selstr) > 1:
        ddGa = scan.ddGa_rel()[1:]
else:
    ddGa = scan.dGsolv_rel()[1:]
mutids = scan.getMutids()[1:]

mutids = np.asarray(mutids)
mutids = mutids[np.where(np.abs(ddGa) >= E)[0]]
mutids = mutids.tolist()
ddGa = np.asarray(ddGa)
ddGa = ddGa[np.where(np.abs(ddGa) >= E)[0]]
ddGa = ddGa.tolist()
n = len(mutids)
seg2sel = dict(('seg%d' % (key + 1), val)
               for (key, val) in enumerate(selstr))

regions = [seg2sel[x.split('_')[0]] for x in mutids]
numbers = [int(x.split('_')[1][1:-1]) for x in mutids]
pdb = pd.parsePQR(pdbfile)

@> 3159 atoms and 1 coordinate sets were parsed in 0.04s.
DEBUG:.prody:3159 atoms and 1 coordinate sets were parsed in 0.04s.


In [8]:
atoms = pdb.select(
    ' and '.join([
        regions[0],
        'resnum %s' % (numbers[0]),
        '(charge >= 0.3 or charge <= -0.3)',
        'sidechain'  # ,
        # 'heavy'
    ])
)
chains = [atoms.getChids()[0]]
resids = [AA_dict[atoms.getResnames()[0]]]

for region, number in zip(regions[1:], numbers[1:]):
    currsel = pdb.select(
        ' and '.join([
            region,
            'resnum %s' % (number),
            '(charge >= 0.3 or charge <= -0.3)',
            'sidechain'  # ,
            # 'heavy'
        ])
    )
    atoms = atoms + currsel
    chains.append(currsel.getChids()[0])
    resids.append(AA_dict[currsel.getResnames()[0]])

# Calc distance matrix
resnums = np.asarray(numbers)
chains = np.asarray(chains)
resids = np.asarray(resids)
lbls = ['%s%d%s' % (rid, res, chid)
        for rid, res, chid in zip(resids, resnums, chains)]
n = len(resnums)

atom_dist = squareform(pdist(atoms.getCoords()))
res_dist = -1.0 * np.ones((n, n))

for i, i_res, i_chid in zip(range(n), resnums, chains):

    i_idx = np.where(np.logical_and(
        atoms.getResnums() == i_res,
        atoms.getChids() == i_chid
    )
    )[0]

    for j, j_res, j_chid in zip(range(n), resnums, chains):
        if i != j:
            j_idx = np.where(np.logical_and(
                atoms.getResnums() == j_res,
                atoms.getChids() == j_chid
            )
            )[0]
            res_dist[i, j] = np.min(atom_dist[np.ix_(i_idx, j_idx)])
        else:
            res_dist[i, j] = 0

In [9]:
# Find edges
res_dist *= np.tri(*res_dist.shape, k=-1)
mask = np.logical_and((res_dist > 0), (res_dist <= cutoff))
row, col = np.where(mask)

# Generate graph
vmax = np.max(np.abs(ddGa))
#norm = mpl.colors.Normalize(vmin=-1 * vmax, vmax=vmax)
#cmap = plt.cm.coolwarm_r

G = nx.Graph()
for node, val, lbl in zip(range(n), ddGa, lbls):
    G.add_node(node, ddGa=val, lbl=lbl, alpha=alpha)  # , size=node_size)
for i, j in zip(row, col):
    G.add_edge(i, j, weight=res_dist[i, j])

In [10]:
pos = nx.spring_layout(G)

In [11]:
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d
        
p=nx.single_source_shortest_path_length(G,ncenter)

In [12]:
import plotly.plotly as py
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
from plotly import tools

In [13]:
edge_trace = go.Scatter(
    x=[], 
    y=[], 
    line=go.Line(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += [x0, x1, None]
    edge_trace['y'] += [y0, y1, None]

node_trace =go.Scatter(
    x=[], 
    y=[], 
    text=[],
    mode='markers', 
    hoverinfo='text',
    marker=go.Marker(
        showscale=True,
        # colorscale options
        # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
        # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
        colorscale='RdBu',
        cmin = -1 * vmax,
        cmax = vmax,
        reversescale=True,
        color=[], 
        size=20,         
        colorbar=dict(
            thickness=15,
            title='ddG (kJ/mol)',
            xanchor='left',
            titleside='right'
        ),
        line=dict(width=2)))

for node in G.nodes():
    x, y = pos[node]
    node_trace['x'].append(x)
    node_trace['y'].append(y)

In [14]:
energy = nx.get_node_attributes(G, 'ddGa')
for node in G.nodes():
    node_trace['marker']['color'].append(energy[node])
    node_info = "{0:.2f}".format(G.node[node]['ddGa'])+ ' kJ/mol'
    node_trace['text'].append(node_info)
    
annotations = go.Annotations()
for k in range(len(lbls)):
    annotations.append(
        go.Annotation(
            text=lbls[k], 
            x=pos[k][0], y=pos[k][1],
            xref='x1', yref='y1',
            font=dict(size=14),
            arrowsize = .4,
            showarrow=True)
    )

In [15]:
fig = go.Figure(data=go.Data([edge_trace, node_trace]),
             layout=go.Layout(
                title='<br>Network Plot',
                titlefont=dict(size=16),
                showlegend=False, 
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=annotations,
                xaxis=go.XAxis(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=go.YAxis(showgrid=False, zeroline=False, showticklabels=False)))

In [68]:
plotly.offline.plot(fig)

'file://F:\\Libraries\\OneDrive - email.ucr.edu\\Work\\Morikis\\Computing\\testing\\barnase_barstar_alascan\\temp-plot.html'

In [17]:
init_notebook_mode(connected=True)
plotly.offline.iplot(fig)

In [ ]:
energy

In [ ]:
G.node

In [ ]:
fig['layout'].update(annotations=annotations2)

In [ ]:
"{0:.2f}".format(13.949999999999999)+ ' kJ/mol'

In [21]:
from plotly.offline.offline import _plot_html
from bs4 import BeautifulSoup

In [46]:
plot_html, plotdivid, width, height = _plot_html(
        fig, {"displayModeBar":"True"}, True, '100%', '100%',False)

In [47]:

soup = BeautifulSoup(plot_html, 'html.parser')
div_text = str(soup.find("div", {"class": "plotly-graph-div"}))
script_text = str(soup.find("script"))

In [49]:
script_text

'<script type="text/javascript">window.PLOTLYENV=window.PLOTLYENV || {};window.PLOTLYENV.BASE_URL="https://plot.ly";Plotly.newPlot("18ce2ebe-e5b7-4b53-b913-c3cb77e2c130", [{"mode": "lines", "hoverinfo": "none", "y": [0.2474670628126142, 0.004610991845476047, null, 0.2474670628126142, 0.41803127966187303, null, 0.2474670628126142, 0.4181019914702241, null, 0.2474670628126142, 0.5383145704578761, null, 0.2019872194374435, 0.41405028150987905, null, 0.2019872194374435, 0.0, null, 0.5949475841310117, 0.41405028150987905, null, 0.4181019914702241, 0.41803127966187303, null, 0.4181019914702241, 0.5383145704578761, null, 0.4181019914702241, 0.6736348910712314, null, 0.8617966488323395, 0.5383145704578761, null, 0.8617966488323395, 0.6736348910712314, null, 0.5383145704578761, 0.6736348910712314, null, 0.5383145704578761, 0.6516503079082471, null, 0.5383145704578761, 0.41803127966187303, null, 0.6736348910712314, 0.6516503079082471, null, 0.6736348910712314, 0.41803127966187303, null, 0.651650

In [31]:
configkeys = (
        'editable',
        'autosizable',
        'fillFrame',
        'frameMargins',
        'scrollZoom',
        'doubleClick',
        'showTips',
        'showLink',
        'sendData',
        'linkText',
        'showSources',
        'displayModeBar',
        'modeBarButtonsToRemove',
        'modeBarButtonsToAdd',
        'modeBarButtons',
        'displaylogo',
        'plotGlPixelRatio',
        'setBackground',
        'topojsonURL'
    )

In [33]:
config = ['displayModeBar']
dict((k, config[k]) for k in configkeys if k in config)

TypeError: list indices must be integers, not str

In [1]:
import pickle as p
import networkx as nx
from scipy.spatial.distance import pdist, squareform
import os
import numpy as np
import prody as pd

In [2]:
alascan = p.load( open( "save.p", "rb" ) )


AESOP: Analysis of Electrostatic Structure of Proteins

Reed E. S. Harrison, Rohith R. Mohan, Dimitrios Morikis
University of California, Riverside; Department of Bioengineering

Correspondence should be directed to Prof. Dimitrios Morikis at
dmorikis@ucr.edu

Copyright (C) 2016  Reed E. S. Harrison, Rohith R. Mohan, Ronald D.
Gorham Jr., Chris A. Kieslich, Dimitrios Morikis

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
ERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.



In [3]:
from aesop_copy import plotNetwork_interactive

In [4]:
plotNetwork_interactive(alascan)

@> 3159 atoms and 1 coordinate sets were parsed in 0.03s.
DEBUG:.prody:3159 atoms and 1 coordinate sets were parsed in 0.03s.
